In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from fastai.vision.widgets import *
from fastai.vision.all import *
import fastai

In [3]:
path = Path('/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess')

In [4]:
file_names = get_image_files(path)

In [5]:
file_names

In [6]:
chesspieces = DataBlock(blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=101),
    get_y=parent_label,
    item_tfms=Resize(128))
dls = chesspieces.dataloaders(path)

In [7]:
dls.valid.show_batch(max_n = 8, nrows=2)

In [8]:
from torchvision.models import resnet50

In [9]:
#fine tune for 3 epochs using pretrained resnet50 
learn = cnn_learner(dls = dls, arch=resnet50, metrics=error_rate)
learn.fine_tune(3, base_lr = 1e-3)

In [10]:
lrs = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [11]:
#unfreeze model and train it for 5 more epochs with changing learning rate
learn.unfreeze()
learn.fit_one_cycle(5, lr_max = slice(1e-4, 1e-3))

In [12]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [13]:
learn.show_results()

In [14]:
interp.plot_top_losses(8, largest= True, nrows=2)

In [15]:
#most confused predictions
interp.most_confused(min_val=5)

In [16]:
#storing the model
learn.export('export.pkl')

In [17]:
# load for model inference
learn_inference = load_learner('export.pkl')

In [18]:
learn_inference.predict('/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess/King/00000004.jpg')

In [19]:
#class order
learn_inference.dls.vocab

In [20]:
#tensor on index 4 has highest pribability (Queen)
learn_inference.predict('/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess/Queen/00000001.jpg')

In [53]:

btn_upload = widgets.FileUpload()
btn_upload

In [78]:
img = PILImage.create(btn_upload.data[-1])

In [79]:

out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [80]:
pred,pred_idx,probs = learn_inference.predict(img)

In [81]:

lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [76]:

btn_run = widgets.Button(description='Classify')
btn_run

In [69]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [82]:

VBox([widgets.Label('Select your chess piece'), 
      btn_upload, btn_run, out_pl, lbl_pred])